In [1]:
import os
os.environ['DEVELOPMENT'] = 'True'
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime

from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement, GroundWater, FieldMngt
from aquacrop.utils import prepare_weather, get_filepath



Running the simulation in development mode.
INFO: No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils


C:\Users\s10034cb\AppData\Roaming\Python\Python39\site-packages\aquacrop\solution\root_zone_water.py:4: NumbaPendingDeprecationWarning: The 'pycc' module is pending deprecation. Replacement technology is being developed.

Pending Deprecation in Numba 0.57.0. For more information please see: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-the-numba-pycc-module
  from numba.pycc import CC


INFO: C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.35.32215\bin\HostX86\x64\cl.exe /c /nologo /O2 /W3 /GL /DNDEBUG /MD -IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.35.32215\include -IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.35.32215\ATLMFC\include -IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Auxiliary\VS\include -IC:\Program Files (x86)\Windows Kits\10\include\10.0.22000.0\ucrt -IC:\Program Files (x86)\Windows Kits\10\\include\10.0.22000.0\\um -IC:\Program Files (x86)\Windows Kits\10\\include\10.0.22000.0\\shared -IC:\Program Files (x86)\Windows Kits\10\\include\10.0.22000.0\\winrt -IC:\Program Files (x86)\Windows Kits\10\\include\10.0.22000.0\\cppwinrt /TcC:\Users\s10034cb\AppData\Local\Temp\tmpgj2dh788\temp.c /FoC:\Users\s10034cb\AppData\Local\Temp\tmpgj2dh788\temp.obj
INFO: C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\1

## Setup comparison function

### Remove headers fn

In [2]:
# drop rows that contain repetitions of the headers throughout the file, one per successive annual run
def remove_headers(data):
    start_points=data[data.isin(['Day']).any(axis=1)].index # begin row drops where 'Day' is in any col
    end_points = [x+3 for x in start_points] # end 3 lines later
    to_drop=[]
    n=0

    while(n < len(start_points)):
        x=list(range(start_points[n]-1, end_points[n]-1)) # adjust drops, one line before the indexes found previously
        to_drop.extend(x) # add all indexes to drop to a list
        n=n+1

    new_data = data.drop(index=to_drop)

    return new_data

### Extract py data fn

In [3]:
def extract_py_data(py_model):
    py_growth=py_model.get_crop_growth()
    py_wflux=py_model.get_water_flux()
    #py_wstorage=py_model.get_water_storage()

    start_year=py_model.sim_start_time.split('/')[0]
    start_month,start_day=py_model.crop.planting_date.split('/')

    py_growth['datetime']=py_growth.apply(lambda x:datetime.datetime(int(x['season_counter']+int(start_year)),int(start_month),int(start_day)+1)+datetime.timedelta(int(x['dap']-1)),axis=1)
    py_wflux['datetime']=py_wflux.apply(lambda x:datetime.datetime(int(x['season_counter']+int(start_year)),int(start_month),int(start_day)+1)+datetime.timedelta(int(x['dap']-1)),axis=1)

    py_growth.drop(['dap','time_step_counter','season_counter'], inplace=True,axis=1)
    py_wflux.drop(['dap','time_step_counter','season_counter'], inplace=True,axis=1)

    py_outputs=pd.merge(py_growth, py_wflux, on='datetime')
    py_outputs['source']='AQ-Py'
    py_outputs.rename(columns={
            'yield_': 'dry_yield',
            # 'Month': 'month',
            # 'Day': 'day',
            },
            inplace=True, 
            errors='raise')

    return py_outputs

    

### Extract Win data fn

In [4]:
def extract_win_data(proj_name):

    file_list=['Crop','Wabal','Prof','Clim'] # ,'CompWC'
    data=[]

    for i in range(0,len(file_list)):
        file_type=file_list[i]
        path='C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+proj_name+file_type+'.OUT'
        if file_type=='Crop':
            temp=pd.read_csv(path,delim_whitespace=True,encoding="ISO-8859-1", skiprows=4, skipfooter=33)
        elif file_type=='Wabal':
            temp=pd.read_csv(path,delim_whitespace=True,encoding="ISO-8859-1", skiprows=4, skipfooter=27)
            temp=temp.drop(['Tr', 'Trx','Tr/Trx', 'WCTot','Rain'],axis=1)
        elif file_type=='Prof':
            temp=pd.read_csv(path,delim_whitespace=True,encoding="ISO-8859-1", skiprows=4, skipfooter=19, index_col=False)
            temp=temp.drop(['Z'],axis=1)
        #elif file_type=='CompWC':
        #    temp=pd.read_csv(path,delim_whitespace=True,encoding="ISO-8859-1", skiprows=5, skipfooter=9)
        elif file_type=='Clim':
            temp=pd.read_csv(path,delim_whitespace=True,encoding="ISO-8859-1", skiprows=4, skipfooter=15)
        
        #if file_type != 'CompWC':
        temp=temp.drop([0])
        
        temp.rename(columns={
            'Year': 'year',
            'Month': 'month',
            'Day': 'day',
            },
            inplace=True, 
            errors='raise'
        )
        temp=remove_headers(temp)
        # add datetime column
        temp['datetime'] = pd.to_datetime(temp[['year','month','day']])
        # convert all cols except date-related cols to floats
        cols=temp.columns.drop(['day','month','year','datetime'])
        temp[cols]=temp[cols].apply(pd.to_numeric, errors='coerce')
        temp=temp.drop(['year','month','day','DAP','Stage'], axis=1)
        data.append(temp)

    temp1=pd.merge(data[0],data[1], on='datetime')
    temp2=pd.merge(temp1,data[2], on='datetime')
    win_data=pd.merge(temp2,data[3], on='datetime')
    #win_data=pd.merge(temp3,data[4], on='datetime')   
    
    # rename cols to align with py col names
    win_data.rename(columns={'CC': 'canopy_cover',
                            'Y(dry)': 'dry_yield',
                            'Biomass': 'biomass',
                            'HI':'harvest_index',
                            'Z':'z_root',
                            },
                    inplace=True, errors='raise'
                    )
    win_data['harvest_index']=win_data['harvest_index']/100
    win_data['canopy_cover']=win_data['canopy_cover']/100
    win_data['biomass']=win_data['biomass']*100
    
    win_data['source']='AQ-Win'

    return win_data
    # Crop: daily crop growth: 4, 33

    # Wabal: daily water balance: 4,27

    # Prof: daily soil-water content in soil profile (WC) and root zone (Wr): 4,19

    # CompWC: daily soil-water content at various depths: 4,9

    # Clim: climate input data: 4,15

### Create datetime

In [3]:
def get_datetime(df, start_year, start_month, start_day):    
    df['year']=df['season_counter'].astype(int)+int(start_year)
    df['combined']=df['year'].astype(str) + ' ' + str(start_month) + ' ' + str(start_day)
    df['start_date']=pd.to_datetime(df['combined'], format='%Y %m %d')
    df['dap-1']=df['dap']-1
    df['time_added']=pd.to_timedelta(df['dap-1'], 'd')
    df['datetime']=df['start_date'] + df['time_added']
    # df.drop(['combined','start_date','start_doy','doy_dap','time_added'],axis=1, inplace=True)
    df=df[(df['dap-1'] >= 0)] # dap starts at 0, anything below 0 is NA
    return(df)

# Run comparison fn

In [4]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [51]:
def run_comparison(win_filename, py_model, target_year=None):

    # Get Python daily output
    py_daily_outputs = py_model.get_crop_growth()
    # py_daily_outputs.rename(columns={'biomass':'biomass_old','canopy_cover':'canopy_cover_old'}, inplace=True) # 'biomass':'biomass_old',
    print(max(py_daily_outputs['biomass']))
    py_daily_outputs['biomass']=py_daily_outputs['biomass']/100
    print(max(py_daily_outputs['biomass']))
    # print(py_daily_outputs.columns)
    py_daily_outputs['canopy_cover']=py_daily_outputs['canopy_cover']*100
    # PY GROWTH COLS = ['time_step_counter', 'season_counter', 'dap', 'gdd', 'gdd_cum',
    #    'z_root', 'canopy_cover', 'canopy_cover_ns', 'biomass', 'biomass_ns',
    #    'harvest_index', 'harvest_index_adj', 'DryYield', 'FreshYield',
    #    'YieldPot']

    py_daily_water=py_model.get_water_flux()
    # PY WATER COLS = ['time_step_counter', 'season_counter', 'dap', 'Wr', 'z_gw',
    #    'surface_storage', 'IrrDay', 'Infl', 'Runoff', 'DeepPerc', 'CR', 'GwIn',
    #    'Es', 'EsPot', 'Tr', 'TrPot']

    # py_daily_storage=py_model.get_water_storage()
    # PY STORAGE COLS = ['time_step_counter', 'growing_season', 'dap', 'th1', 'th2', 'th3',
    #    'th4', 'th5', 'th6', 'th7', 'th8', 'th9', 'th10', 'th11', 'th12']

    start_year=int(py_model.sim_start_time.split('/')[0])
    start_month,start_day=py_model.crop.planting_date.split('/')

    py_daily_outputs=get_datetime(py_daily_outputs, start_year,start_month,start_day)

    py_daily_water=get_datetime(py_daily_water, start_year,start_month,start_day)

    # py_daily_storage=get_datetime(py_daily_storage, start_year,start_month,start_day)
    
    # py_daily_outputs.to_csv('py_daily_outputs.csv', index = False, encoding='utf-8')
    # py_daily_water.to_csv('py_daily_water.csv', index = False, encoding='utf-8')
    py_outputs=pd.merge(py_daily_outputs, py_daily_water, on='datetime')
    # py_outputs=pd.merge(py_outputs, py_daily_storage,on='datetime')

    py_outputs['source']='AQ-Py'
    py_outputs.to_csv('py_outputs.csv', index = False, encoding='utf-8')
    # print(py_outputs.columns)
    # print(py_outputs.head())

    # Get Python annual outputs
    py_annual_outputs = py_model._outputs.final_stats.round(3)
    py_annual_outputs.columns = ["Season","CropType","datetime","HarvestDate (Step)","dry_yield", "fresh_yield","yield_potential", "Seasonal irrigation (mm)"]
    # py_annual_outputs.columns = ["Season","CropType","datetime","HarvestDate (Step)","dry_yield", "Seasonal irrigation (mm)"]
    # py_annual_outputs['date'] = pd.to_datetime(py_annual_outputs['datetime'],format='%Y%m%d')
    # py_annual_outputs['year'] = pd.DatetimeIndex(py_annual_outputs['date']).year
    # py_annual_outputs['year'] = py_annual_outputs['year'] - 1
    # py_annual_outputs['year'] = py_annual_outputs['year'].astype(str)
    py_annual_outputs['source']='Py'

    # Get windows daily outputs (CROP)
    win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT', 
                            skiprows=4, skipfooter=33,  delim_whitespace=True,encoding="latin1")
    # win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV60/AquaCropV60/OUTP/'+win_filename+'Crop.OUT', 
    #                         skiprows=4, skipfooter=33,  delim_whitespace=True,encoding="latin1")
    # WIN DAILY CROP COLS = ['Day', 'Month', 'Year', 'DAP', 'Stage', 'GD', 'Z', 'StExp', 'StSto',
    #    'StSen', 'StSalt', 'StWeed', 'CC', 'CCw', 'StTr', 'Kc(Tr)', 'Trx', 'Tr',
    #    'TrW', 'Tr/Trx', 'WP', 'Biomass', 'HI', 'Y(dry)', 'Y(fresh)',
    #    'Brelative', 'WPet', 'Bin', 'Bout']
    win_daily_outputs=win_daily_outputs.drop([0])
    win_daily_outputs.rename(columns={
        'Year': 'year',
        'Month': 'month',
        'Day': 'day',
        },
        inplace=True, 
        errors='raise')
    win_daily_outputs=remove_headers(win_daily_outputs)

    # Get windows daily water outputs (WABAL)
    win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT', 
                            skiprows=4, skipfooter=27,  delim_whitespace=True,encoding="latin1")
    # win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV60/AquaCropV60/OUTP/'+win_filename+'WABAL.OUT', 
    #                         skiprows=4, skipfooter=27,  delim_whitespace=True,encoding="latin1")
     # WIN DAILY WABAL COLS = ['Day', 'Month', 'Year', 'DAP', 'Stage', 'WCTot', 'Rain', 'Irri', 'Surf',
    #    'Infilt', 'RO', 'Drain', 'CR', 'Zgwt', 'Ex', 'E', 'E/Ex', 'Trx', 'Tr',
    #    'Tr/Trx', 'ETx', 'ET', 'ET/ETx']
    win_daily_water=win_daily_water.drop([0])
    win_daily_water.rename(columns={
        'Year': 'year',
        'Month': 'month',
        'Day': 'day',
        },
        inplace=True, 
        errors='raise')
    win_daily_water=remove_headers(win_daily_water)
    win_daily_water.drop(['Tr'],axis=1,inplace=True) # remove Tr because it's a duplicate of win_daily_outputs version

    def line_fixer(x):
        if not x[0].isnumeric() and x[-1].isnumeric():
            return x[1:] 
        
        if not x[-1].isnumeric() and x[0].isnumeric():
            return x[:-1]
        
        return None

    # Get windows daily soil profile outputs (Prof)
    win_daily_profile = pd.read_csv('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Prof.OUT', 
                            skiprows=4, skipfooter=20,  delim_whitespace=True, encoding="latin1", on_bad_lines=line_fixer, engine='python')
    # win_daily_profile = pd.read_csv('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV60/AquaCropV60/OUTP/'+win_filename+'Prof.OUT', 
    #                         skiprows=4, skipfooter=20,  delim_whitespace=True, encoding="latin1", on_bad_lines=line_fixer, engine='python')
     # WIN DAILY PROFILE COLS = ['Day', 'Month', 'Year', 'DAP', 'Stage', 'WCTot', 'Wr(Zx)', 'Z', 'Wr',
    #    'Wr(SAT)', 'Wr(FC)', 'Wr(exp)', 'Wr(sto)', 'Wr(sen)', 'Wr(PWP)']
    win_daily_profile=win_daily_profile.drop([0])
    win_daily_profile.rename(columns={
        'Year': 'year',
        'Month': 'month',
        'Day': 'day',
        },
        inplace=True, 
        errors='raise')
    win_daily_profile=remove_headers(win_daily_profile)
    # win_daily_profile.to_csv('win_daily_profile.csv', index = False, encoding='utf-8')
    # print(win_daily_water.columns)
    win_daily_profile.drop(['Z'],axis=1,inplace=True) # remove Z because it's a duplicate of win_daily_outputs version

    # Get windows daily soil water storage
    # win_daily_storage = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WC.OUT', 
    #                         skiprows=4, skipfooter=9,  delim_whitespace=True,encoding="latin1")
    # win_daily_storage = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV60/AquaCropV60/OUTP/'+win_filename+'WC.OUT', 
    #                         skiprows=4, skipfooter=9,  delim_whitespace=True,encoding="latin1")
    # add datetime columns
    win_daily_outputs['datetime'] = pd.to_datetime(win_daily_outputs[['year','month','day']])
    win_daily_water['datetime'] = pd.to_datetime(win_daily_water[['year','month','day']])
    win_daily_profile['datetime'] = pd.to_datetime(win_daily_profile[['year','month','day']])
    # win_daily_storage['datetime'] = pd.to_datetime(win_daily_storage[['year','month','day']])

    # convert all cols except date-related cols to floats
    cols=win_daily_outputs.columns.drop(['day','month','year','datetime'])
    cols_water=win_daily_water.columns.drop(['day','month','year','datetime'])
    cols_profile=win_daily_profile.columns.drop(['day','month','year','datetime'])
    # cols_storage=win_daily_storage.columns.drop(['day','month','year','datetime'])

    win_daily_outputs[cols]=win_daily_outputs[cols].apply(pd.to_numeric, errors='coerce')
    win_daily_water[cols_water]=win_daily_water[cols_water].apply(pd.to_numeric, errors='coerce')
    win_daily_profile[cols_profile]=win_daily_profile[cols_profile].apply(pd.to_numeric, errors='coerce')
    # win_daily_storage[cols_storage]=win_daily_storage[cols_storage].apply(pd.to_numeric, errors='coerce')

    win_temp=pd.merge(win_daily_outputs,win_daily_water,on='datetime')
    # win_temp2=pd.merge()
    win_outputs=pd.merge(win_temp,win_daily_profile, on='datetime')
    # rename cols to align with py col names
    win_outputs.rename(columns={'CC': 'canopy_cover',
                                'Y(dry)': 'Yield',
                                # 'YieldPart' : 'Yield',
                                'Biomass': 'biomass',
                                'HI':'harvest_index',
                                'Zgwt':'z_gw',
                                'Infilt':'Infl',
                                'Z':'z_root',
                                'Irri':'IrrDay',
                                'E':'Es',
                                'Surf':'surface_storage'
                                },
                    inplace=True, errors='raise')
    win_outputs['harvest_index']=win_outputs['harvest_index']/100
    win_outputs['source']='AQ-Win'
    win_outputs.to_csv('win_outputs.csv', index = False, encoding='utf-8')

    # Get windows annual outputs
    # V7 READING
    win_annual_outputs=pd.read_csv('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Harvest.OUT', 
                                skiprows=4, skipfooter=8,  delim_whitespace=True,encoding="ISO-8859-1")
    win_annual_outputs=win_annual_outputs.drop([0])
    win_annual_outputs=remove_headers(win_annual_outputs)
    win_annual_outputs = win_annual_outputs.iloc[: , :-5] # remove 5 blank cols due to reading error
    win_annual_outputs['datetime']=pd.to_datetime(win_annual_outputs[['Year','Month','Day']])
    win_annual_outputs.columns=['Nr','Day','Month','Year','biomass','dry_yield','fresh-yield', 'datetime']
    win_annual_outputs=win_annual_outputs[win_annual_outputs['Nr'] != '0'] # remove blank rows
    win_annual_outputs['source']='Win'

    # V6 READING
    # win_annual_outputs=pd.read_csv('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV60/AquaCropV60/OUTP/'+win_filename+'Run.OUT', 
    #                             skiprows=3, skipfooter=40,  delim_whitespace=True,encoding="ISO-8859-1")
    # win_annual_outputs=win_annual_outputs.drop([0])
    # win_annual_outputs=remove_headers(win_annual_outputs)
    # win_annual_outputs = win_annual_outputs.iloc[: , :-5] # remove 5 blank cols due to reading error
    # win_annual_outputs.rename(columns={
    #     'Year1': 'year',
    #     'Month1': 'month',
    #     'Day1': 'day',
    #     'Yield' : 'dry_yield',
    #     },
    #     inplace=True, 
    #     errors='raise')
    # win_annual_outputs['datetime']=pd.to_datetime(win_annual_outputs[['year','month','day']])
    # print(win_annual_outputs.head())
    # win_annual_outputs.columns=['Nr','Day','Month','Year','biomass','dry_yield','fresh-yield', 'datetime']
    # win_annual_outputs=win_annual_outputs[win_annual_outputs['Nr'] != '0'] # remove blank rows
    # win_annual_outputs['source']='Win'
 
    # Combine annual outputs
    combined_annual=pd.concat([win_annual_outputs, py_annual_outputs])
    combined_annual['dry_yield']=combined_annual['dry_yield'].astype(float)
    combined_annual.to_csv('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/aquacrop/outputs/' + win_filename + '_annual_outputs.csv', index = False, encoding='utf-8')
    # win_outputs.to_csv('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/aquacrop/outputs/' + win_filename + 'win_daily_outputs.csv', index = False, encoding='utf-8')
    # combined_annual.drop(['datetime','day','month'],axis=1,inplace=True)

    # custom year
    if target_year:
        date_1= str(target_year-1) + '-05-01'
        date_2= str(target_year) + '-04-30'

    # manipulating for specific plot
    temp_a = py_outputs[['datetime', 'Wr']]
    temp_a.rename(columns={'Wr' : 'Py_Wr'}, inplace=True)
    temp_b = win_outputs[['datetime', 'Wr']]
    temp_b.rename(columns={'Wr' : 'Win_Wr'}, inplace=True)
    temp_c = pd.merge(temp_a, temp_b, on='datetime', how='left')
    diff_series=pd.Series(temp_c['Py_Wr'].sub(temp_c['Win_Wr']), name='Wr_diff')
    temp_d = pd.concat([temp_c['datetime'], diff_series.to_frame()], axis=1)
    if target_year:
        temp_d=temp_d[(temp_d['datetime'] < date_2) & (temp_d['datetime'] > date_1)]
        temp_e=win_outputs[(win_outputs['datetime'] < date_2) & (win_outputs['datetime'] > date_1)]
        # sns.lineplot(data=temp_e, x='datetime', y='CR')
    # sns.lineplot(data=temp_d, x='datetime', y='Wr_diff')
    
    # plt.show()

    # Combine daily outputs
    both_daily_outputs=pd.concat([win_outputs, py_outputs])
    both_daily_outputs.to_csv('both_daily_outputs.csv', index = False, encoding='utf-8')
    if target_year:
        both_daily_outputs=both_daily_outputs[(both_daily_outputs['datetime'] < date_2) & (both_daily_outputs['datetime'] > date_1)]

    fig1, ax =plt.subplots(ncols=2, nrows=2, figsize=(10,8))


    sns.lineplot(data=both_daily_outputs, x='datetime',y='harvest_index', hue='source', ax=ax[0,0]).get_legend().remove() # , ax=ax[0,0] , dashes=dash_style, style='source'
    sns.lineplot(data=combined_annual, x='datetime', y='dry_yield', hue='source', ax=ax[1,1]) # .reset_index()
    sns.lineplot(data=both_daily_outputs, x='datetime',y='canopy_cover', hue='source', ax=ax[0,1]).get_legend().remove() # , ax=ax[0,0]
    sns.lineplot(data=both_daily_outputs, x='datetime',y='biomass', hue='source', ax=ax[1,0]).get_legend().remove()#, ax=ax[1,0]
    plt.show()
    
    fig2,ax2=plt.subplots(ncols=2,nrows=5,figsize=(10,8))
    sns.lineplot(data=both_daily_outputs, x='datetime',y='IrrDay', hue='source', ax=ax2[0,0]).get_legend().remove()#, ax=ax[0,1]
    sns.lineplot(data=both_daily_outputs, x='datetime',y='Es', hue='source', ax=ax2[0,1]).get_legend().remove() # , ax=ax[0,0]
    sns.lineplot(data=both_daily_outputs, x='datetime',y='Tr', hue='source', ax=ax2[1,1]).get_legend().remove()#, ax=ax[1,1]
    sns.lineplot(data=both_daily_outputs, x='datetime',y='z_gw', hue='source', ax=ax2[1,0]).get_legend().remove()
    sns.lineplot(data=both_daily_outputs, x='datetime',y='CR', hue='source', ax=ax2[2,0]).get_legend().remove()
    sns.lineplot(data=both_daily_outputs, x='datetime',y='Infl', hue='source', ax=ax2[2,1]).get_legend().remove()
    sns.lineplot(data=both_daily_outputs, x='datetime',y='Wr', hue='source', ax=ax2[3,0]).get_legend().remove()
    sns.lineplot(data=both_daily_outputs, x='datetime',y='z_root', hue='source', ax=ax2[3,1]).get_legend().remove()
    sns.lineplot(data=both_daily_outputs, x='datetime',y='surface_storage', hue='source', ax=ax2[4,0]).get_legend().remove()
    plt.show()

    #save all_daily_outputs
    python_output(py_model, win_filename)

## Save output fn

In [6]:
def python_output(model, name):
    # get model outputs
    model_growth = model.get_crop_growth()
    model_water = model.get_water_flux()
    model_storage = model.get_water_storage()

    # # get save path
    base_path = 'C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/aquacrop/outputs/'
    sp_path = base_path + name
    growth_path = sp_path + '_growth.csv'
    water_path = sp_path + '_water.csv'
    storage_path = sp_path + '_storage.csv'

    # # save outputs
    model_growth.to_csv(growth_path, index = False, encoding='utf-8')
    model_water.to_csv(water_path, index = False, encoding='utf-8')
    model_storage.to_csv(storage_path, index = False, encoding='utf-8')

    # # get start year and date
    start_year=int(model.sim_start_time.split('/')[0])
    start_month,start_day=model.crop.planting_date.split('/')

    # # save starting date info
    start_info = pd.DataFrame(data = {'date_info' : [str(start_day) + '-' + str(start_month) + '-' + str(start_year)]})
    # print(start_info)
    start_info.to_csv(sp_path + '_date.csv', index = False, encoding='utf-8')

# Exercise 7

## Exercise 7.1 with Tunis Local soil

In [52]:
# Tunis climate
filepath=get_filepath('tunis_climate.txt')
tunis_weather = prepare_weather(filepath)

# Local Tunis soil
tunis_soil = Soil(soil_type='ac_TunisLocal')

# crops
wheat = Crop('WheatGDD', planting_date='10/15')

# IWC
wet_dry = InitialWaterContent(wc_type='Num',
                              method='Depth',
                              depth_layer=[0.3,0.9],
                              value=[0.3,0.15])

# irr management
irr_mngt = IrrigationManagement(irrigation_method=0)

model = AquaCropModel(sim_start_time=f'{1979}/10/15',
                      sim_end_time=f'{2002}/03/31',
                      weather_df=tunis_weather,
                      soil=tunis_soil,
                      crop=wheat,
                      irrigation_management=irr_mngt,
                      initial_water_content=wet_dry,
                      )
model.run_model(till_termination=True)

True

In [8]:
# python_output(model, 'Tun_WW_LocalSoil')

In [8]:
run_comparison('Tun_WW_LocalSoil3',model)

1904.7095936377193
19.047095936377193


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\3743229160.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\3743229160.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\3743229160.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid t

### Exercise 7.1 with Sandy Loam

In [10]:
# Sandy loam soil
sandy_loam = Soil(soil_type='SandyLoam')

model = AquaCropModel(sim_start_time=f'{1979}/10/15',
                      sim_end_time=f'{2002}/03/30',
                      weather_df=tunis_weather,
                      soil=sandy_loam,
                      crop=wheat,
                      irrigation_management=irr_mngt,
                      initial_water_content=wet_dry,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Tun_WW_SLSoil')
run_comparison('Tun_WW_SLSoil3', model)

1922.0250874810938
19.220250874810937


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

## Exercise 7.2 Local wheat variety

In [49]:
# Local wheat variety
local_wheat=Crop('WheatLongGDD', planting_date= '10/15')

model = AquaCropModel(sim_start_time=f'{1979}/10/15',
                      sim_end_time=f'{2002}/05/31',
                      weather_df=tunis_weather,
                      soil=sandy_loam,
                      crop=local_wheat,
                      irrigation_management=irr_mngt,
                      initial_water_content=wet_dry,
                      )
model.run_model(till_termination=True)

python_output(model, 'Tun_WW_LongVar')
run_comparison('Tun_WW_LongVar2', model)

2301.831093377243
23.018310933772433


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

In [50]:
model.get_simulation_results()

,Season,crop Type,Harvest Date (YYYY/MM/DD),Harvest Date (Step),Dry yield (tonne/ha),Fresh yield (tonne/ha),Yield potential (tonne/ha),Seasonal irrigation (mm)
0,0,WheatLongGDD,1980-06-12,240,9.482092,94.820924,10.795086,0
1,1,WheatLongGDD,1981-06-09,602,7.324720,73.247196,10.172598,0
2,2,WheatLongGDD,1982-06-04,962,10.172808,101.728083,10.651749,0
3,3,WheatLongGDD,1983-06-01,1324,6.973368,69.733684,9.828693,0
4,4,WheatLongGDD,1984-06-10,1699,8.010311,80.103112,10.635433,0
5,5,WheatLongGDD,1985-06-05,2059,10.077800,100.778000,10.635419,0
6,6,WheatLongGDD,1986-05-28,2416,9.255992,92.559917,10.527221,0
7,7,WheatLongGDD,1987-06-11,2795,8.849079,88.490792,10.873153,0
8,8,WheatLongGDD,1988-05-21,3140,6.732839,67.328393,10.499824,0
9,9,WheatLongGDD,1989-06-01,3516,7.477476,74.774760,10.853154,0


## Exercise 7.3 Different initial conditions

WetDry

In [12]:
irr_mngt = IrrigationManagement(irrigation_method=0)

model = AquaCropModel(sim_start_time=f'{1979}/10/15',
                      sim_end_time=f'{2002}/05/31',
                      weather_df=tunis_weather,
                      soil=sandy_loam,
                      crop=wheat,
                      irrigation_management=irr_mngt,
                      initial_water_content=wet_dry,
                      )
model.run_model(till_termination=True)

python_output(model, 'Tun_WW_WetDry')
run_comparison('Tun_WW_WetDry', model)

1922.0250874810938
19.220250874810937


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

Field Capacity:

In [13]:
# Change initial water content 
field_capacity = InitialWaterContent(value=['FC'])

model = AquaCropModel(sim_start_time=f'{1979}/10/15',
                      sim_end_time=f'{2002}/05/31',
                      weather_df=tunis_weather,
                      soil=sandy_loam,
                      crop=wheat,
                      irrigation_management=irr_mngt,
                      initial_water_content=field_capacity,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Tun_WW_FC')
run_comparison('Tun_WW_FC', model)

1922.0250874810938
19.220250874810937


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

30% TAW

In [14]:
# Change initial water content
iwc30taw = InitialWaterContent('Pct','Layer',[1],[30])

model = AquaCropModel(sim_start_time=f'{1979}/10/15',
                      sim_end_time=f'{2002}/05/31',
                      weather_df=tunis_weather,
                      soil=sandy_loam,
                      crop=wheat,
                      irrigation_management=irr_mngt,
                      initial_water_content=iwc30taw,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Tun_WW_30TAW')
run_comparison('Tun_WW_30TAW', model)

1869.415122304942
18.69415122304942


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

75% TAW

In [15]:
# Change initial water content
iwc75taw = InitialWaterContent('Pct','Layer',[1],[75])

model = AquaCropModel(sim_start_time=f'{1979}/10/15',
                      sim_end_time=f'{2002}/05/31',
                      weather_df=tunis_weather,
                      soil=sandy_loam,
                      crop=wheat,
                      irrigation_management=irr_mngt,
                      initial_water_content=iwc75taw,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Tun_WW_75TAW')
run_comparison('Tun_WW_75TAW', model)

1921.9419451101605
19.219419451101604


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

Wilting Point

In [16]:
# Change initial water content
iwc_wp = InitialWaterContent('Prop','Layer',[1],['WP'])
iwc_wp2 = InitialWaterContent('Pct','Layer',[1],[10])

irr_mngt = IrrigationManagement(irrigation_method=0)

model = AquaCropModel(sim_start_time=f'{1979}/10/15',
                      sim_end_time=f'{2002}/05/31',
                      weather_df=tunis_weather,
                      soil=sandy_loam,
                      crop=wheat,
                      irrigation_management=irr_mngt,
                      initial_water_content=iwc_wp,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Tun_WW_WP')
run_comparison('Tun_WW_WP', model)

1898.3484908506846
18.983484908506846


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

## Exercise 7.6 Net irrigation requirement

In [17]:
# Exercise states 30% of RAW, that's what I have input into the irrigation file in AQ-Win
# AQ-Py only deals with TAW, not RAW, so a conversion must be done.
# Conversion is based on the p_up2 attribute of crop, reflects proportion of TAW that is RAW.
# In this case (WheatGDD) has p_up2 of 0.65 so 30 * 0.65 = 19.5, find inverse: 100 - 19.5 = 80.5
net_irr = IrrigationManagement(irrigation_method=4,NetIrrSMT=80.5, MaxIrr=50)
wp = InitialWaterContent(value=['WP'])
wheat_dec = Crop('WheatGDD',planting_date='12/01')

model = AquaCropModel(sim_start_time=f'{1979}/12/01',
                      sim_end_time=f'{2002}/05/31',
                      weather_df=tunis_weather,
                      soil=sandy_loam,
                      crop=wheat_dec,
                      irrigation_management=net_irr,
                      initial_water_content=wp,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Tun_WW_NetIrr')
run_comparison('Tun_WW_NetIrr3', model)

1804.5095943584206
18.045095943584204


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

## Exercise 7.7 Assess deficit irrigation schedules

In [18]:
all_1_decs=pd.date_range('1979/12/01', '2001/12/01',freq='12MS')

dates=[]

for each_start in all_1_decs:
    app1=each_start
    app2=each_start+pd.Timedelta(31,'d')
    app3=each_start+pd.Timedelta(61,'d')
    dates.extend([app1,app2,app3])

n_years=len(all_1_decs)
depths=[30,40,40]*n_years

schedule=pd.DataFrame([dates,depths]).T # create pandas DataFrame
schedule.columns=['Date','Depth'] # name columns

print(schedule)

net_irr = IrrigationManagement(irrigation_method=3, Schedule=schedule, MaxIrr = 50)
wp = InitialWaterContent(wc_type='Pct',method='Layer',depth_layer=[1],value=[10])
wp2 = InitialWaterContent(wc_type='Prop',method='Layer',depth_layer=[1],value=['WP'])
wheat_dec = Crop('WheatGDD',planting_date='12/01')
sandy_loam = Soil(soil_type='SandyLoam')

model = AquaCropModel(sim_start_time=f'{1979}/12/01',
                      sim_end_time=f'{2002}/05/25',
                      weather_df=tunis_weather,
                      soil=sandy_loam,
                      crop=wheat_dec,
                      irrigation_management=net_irr,
                      initial_water_content=wp2,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Tun_WW_Deficit')
run_comparison('Tun_WW_Deficit3', model)

         Date Depth
0  1979-12-01    30
1  1980-01-01    40
2  1980-01-31    40
3  1980-12-01    30
4  1981-01-01    40
..        ...   ...
64 2001-01-01    40
65 2001-01-31    40
66 2001-12-01    30
67 2002-01-01    40
68 2002-01-31    40

[69 rows x 2 columns]
1790.6198121568323
17.906198121568323


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

With field capacity:

# Exercise 8

## Exercise 8.2 Paddy rice in Hyderabad planting dates

In [19]:
# NO BUNDS TEST

# Hyderabad climate
filepath=get_filepath('hyderabad_climate.txt')
hyderabad_weather = prepare_weather(filepath)

# Hyderabad soil
paddy_soil = Soil(soil_type='Paddy')

# crop
local_rice = Crop('localpaddy', planting_date='08/01')

# IWC
fc = InitialWaterContent(value=['FC'])

# irr management
irr_mngt = IrrigationManagement(irrigation_method=0)

model = AquaCropModel(sim_start_time=f'{2000}/08/01',
                      sim_end_time=f'{2010}/12/31',
                      weather_df=hyderabad_weather,
                      soil=paddy_soil,
                      crop=local_rice,
                      irrigation_management=irr_mngt,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Hyd_Rice_1Aug_noBunds')
run_comparison('Hyd_Rice_1Aug_noBunds', model)

1491.7549387586002
14.917549387586002


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

In [20]:
# Hyderabad climate
filepath=get_filepath('hyderabad_climate.txt')
hyderabad_weather = prepare_weather(filepath)

# Hyderabad soil
paddy_soil = Soil(soil_type='Paddy')

# crop
local_rice = Crop('localpaddy', planting_date='08/01')

# IWC
fc = InitialWaterContent(value=['FC'])

# irr management
irr_mngt = IrrigationManagement(irrigation_method=0)

# field management
bunds20 = FieldMngt(bunds=True, z_bund=0.20)

model = AquaCropModel(sim_start_time=f'{2000}/08/01',
                      sim_end_time=f'{2010}/12/31',
                      weather_df=hyderabad_weather,
                      soil=paddy_soil,
                      crop=local_rice,
                      field_management=bunds20,
                      irrigation_management=irr_mngt,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Hyd_Rice_1Aug')
run_comparison('Hyd_Rice_1Aug3', model)

1625.5004211089204
16.255004211089204


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

In [21]:
# NO BUNDS TEST

# crop
local_rice = Crop('localpaddy', planting_date='07/15')

model = AquaCropModel(sim_start_time=f'{2000}/07/15',
                      sim_end_time=f'{2010}/12/31',
                      weather_df=hyderabad_weather,
                      soil=paddy_soil,
                      crop=local_rice,
                      irrigation_management=irr_mngt,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Hyd_Rice_15Jul_noBunds')
run_comparison('Hyd_Rice_15Jul_noBunds', model)

1504.8963594486308
15.048963594486308


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

In [22]:
# crop
local_rice = Crop('localpaddy', planting_date='07/15')

model = AquaCropModel(sim_start_time=f'{2000}/07/15',
                      sim_end_time=f'{2010}/12/31',
                      weather_df=hyderabad_weather,
                      soil=paddy_soil,
                      crop=local_rice,
                      field_management=bunds20,
                      irrigation_management=irr_mngt,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Hyd_Rice_15Jul')
run_comparison('Hyd_Rice_15Jul_check', model)

1604.5281788826824
16.045281788826824


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

## Exercise 8.3 Initial conditions

Field capacity

In [23]:
# Hyderabad climate
filepath=get_filepath('hyderabad_climate.txt')
hyderabad_weather = prepare_weather(filepath)

# Local Brussels soil
paddy_soil = Soil(soil_type='Paddy')

# crop
local_rice = Crop('localpaddy', planting_date='08/01')

# IWC
fc = InitialWaterContent(value=['FC'])

# irr management
irr_mngt = IrrigationManagement(irrigation_method=0)

model = AquaCropModel(sim_start_time=f'{2000}/08/01',
                      sim_end_time=f'{2010}/12/31',
                      weather_df=hyderabad_weather,
                      soil=paddy_soil,
                      crop=local_rice,
                      irrigation_management=irr_mngt,
                      field_management=bunds20,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Hyd_Rice_FC')
run_comparison('Hyd_Rice_FC3', model)

1625.5004211089204
16.255004211089204


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

30% TAW

In [24]:
# IWC
taw30 = InitialWaterContent('Pct','Layer',[1,2],[30,30])
model = AquaCropModel(sim_start_time=f'{2000}/01/01',
                      sim_end_time=f'{2010}/12/31',
                      weather_df=hyderabad_weather,
                      soil=paddy_soil,
                      crop=local_rice,
                      irrigation_management=irr_mngt,
                      field_management=bunds20,
                      initial_water_content=taw30,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Hyd_Rice_30TAW')
run_comparison('Hyd_Rice_30TAW', model)

1624.2501984040168
16.242501984040167


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

75% TAW

In [25]:
# IWC
taw75 = InitialWaterContent('Pct','Layer',[1,2],[75,75])

model = AquaCropModel(sim_start_time=f'{2000}/01/01',
                      sim_end_time=f'{2010}/12/31',
                      weather_df=hyderabad_weather,
                      soil=paddy_soil,
                      crop=local_rice,
                      field_management=bunds20,
                      irrigation_management=irr_mngt,
                      initial_water_content=taw75,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Hyd_Rice_75TAW')
run_comparison('Hyd_Rice_75TAW', model)

1631.4289806017036
16.314289806017037


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

Wet Top

In [26]:
# IWC
wetTop = InitialWaterContent('Prop','Depth',[0.5,2],['FC','WP'])
wetTopManual = InitialWaterContent('Pct', 'Depth',[0.5,2], [50, 30])

model = AquaCropModel(sim_start_time=f'{2000}/08/01',
                      sim_end_time=f'{2010}/12/31',
                      weather_df=hyderabad_weather,
                      soil=paddy_soil,
                      crop=local_rice,
                      field_management=bunds20,
                      irrigation_management=irr_mngt,
                      initial_water_content=wetTopManual,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Hyd_Rice_wetTop')
run_comparison('Hyd_Rice_wetTop', model)

1631.8057827946927
16.318057827946927


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

## Exercise 8.7 Irr. req. with shallow groundwater table

### a\) 1m groundwater

In [27]:
# Hyderabad climate
filepath=get_filepath('hyderabad_climate.txt')
hyderabad_weather = prepare_weather(filepath)

# Exercise states 30% of RAW, that's what I have input into the irrigation file in AQ-Win
# AQ-Py only deals with TAW, not RAW, so a conversion must be done.
# Conversion is based on the p_up2 attribute of crop, reflects proportion of TAW that is RAW.
# BUT Win works from top down, whereas Py works from bottom up when measuring SMT
# In this case (WheatGDD) has p_up2 of 0.65 so 30 * 0.65 = 0.195
# Then invert the percentage to get bottom-up value: 1 - 0.195 = 80.5 % of TAW
net_irr = IrrigationManagement(irrigation_method=4,NetIrrSMT=80.5)
clay_loam = Soil(soil_type='ClayLoam')
wheat_nov = Crop('HydWheatGDD', planting_date='11/01')
fc = InitialWaterContent(value=['FC'])
groundwater_1m = GroundWater('Y','Constant',dates=[f'{2000}/09/01'], values=[1])

model = AquaCropModel(sim_start_time=f'{2000}/09/01',
                      sim_end_time=f'{2010}/12/31',
                      weather_df=hyderabad_weather,
                      soil=clay_loam,
                      crop=wheat_nov,
                      irrigation_management=net_irr,
                      initial_water_content=fc,
                      groundwater=groundwater_1m,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Hyd_W_NetIrr_GW1m')
run_comparison('Hyd_W_NetIrr_GW1m_2', model)

1747.4100688628628
17.474100688628628


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

### b\) 2m groundwater

In [28]:
groundwater_2m = GroundWater('Y','Constant',dates=[f'{2000}/09/01'], values=[2])

model = AquaCropModel(sim_start_time=f'{2000}/09/01',
                      sim_end_time=f'{2010}/12/31',
                      weather_df=hyderabad_weather,
                      soil=clay_loam,
                      crop=wheat_nov,
                      irrigation_management=net_irr,
                      initial_water_content=fc,
                      groundwater=groundwater_2m,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Hyd_W_NetIrr_GW2m_fixedStart')
run_comparison('Hyd_W_NetIrr_GW2m_2', model)

1747.4100688628628
17.474100688628628


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

# Exercise 9

## Exercise 9.1 Potato in Brussels

In [29]:
# Brussels climate
filepath=get_filepath('brussels_climate.txt')
brussels_weather = prepare_weather(filepath)

# Local Brussels soil
loam_soil = Soil(soil_type='Loam')

# crops
potato = Crop('PotatoLocalGDD', planting_date='04/25')

# IWC
fc = InitialWaterContent(value=['FC'])

# irr management
irr_mngt = IrrigationManagement(irrigation_method=0)

model = AquaCropModel(sim_start_time=f'{1976}/04/25', # 1976
                      sim_end_time=f'{2005}/12/31', # 2005
                      weather_df=brussels_weather,
                      soil=loam_soil,
                      crop=potato,
                      irrigation_management=irr_mngt,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

python_output(model, 'Bru_Pot')
run_comparison('Bru_Pot3', model)

#daily_outputs = run_comparison_v7('Bru_Pot', model)

1390.9983956694628
13.909983956694628


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

## 9.4 Net irrigation requirement

In [30]:
# Exercise states 35% of RAW, that's what I have input into the irrigation file in AQ-Win
# AQ-Py only deals with TAW, not RAW, so a conversion must be done.
# Conversion is based on the p_up2 attribute of crop, reflects proportion of TAW that is RAW.
# In this case (PotatoLocalGDD) has p_up2 of 0.6 so 35 * 0.6 = 21

net_irr = IrrigationManagement(irrigation_method=4,NetIrrSMT=79, MaxIrr=40)

model = AquaCropModel(sim_start_time=f'{1976}/03/01',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loam_soil,
                      crop=potato,
                      irrigation_management=net_irr,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Bru_Pot_NetIrr2')
run_comparison('Bru_Pot_NetIrr2', model)

1451.4529795565302
14.514529795565302


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

## 9.5 Generating irrigation schedules

In [31]:
# function to return the irrigation depth to apply on next day
def get_depth(model, taw):    
    t = model._clock_struct.time_step_counter # current timestep
    # print(f'Depletion = {model._init_cond.depletion}, model TAW = {model._init_cond.taw}, target TAW: {taw}')
    # if t>0 and model._init_cond.depletion/model._init_cond.taw > taw:
    if t>0 and model._init_cond.taw < taw:
        depth=15
    else:
        depth=0

    return depth

Rainfed test

In [32]:
# Loamy sand soil
loamy_sand = Soil(soil_type='LoamySand')

# crop
potato = Crop('PotatoLocalGDD', planting_date='04/25')

# create model with IrrMethod= Constant depth
model = AquaCropModel(sim_start_time=f'{1976}/04/25',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loamy_sand,
                      crop=potato,
                      irrigation_management=IrrigationManagement(irrigation_method=0),
                      initial_water_content=fc,
                      )
                       
model.run_model(till_termination=True)
# model._initialize()

# while model._clock_struct.model_is_finished is False:    
#     # get depth to apply, RAW = 36%, p_up2 = 0.6 so TAW = 21.6, inverse is 78.4
#     depth=get_depth(model,78.4)
    
#     model._param_struct.IrrMngt.depth=depth

#     model.run_model(initialize_model=False)

# python_output(model, 'Bru_Pot_Rainfed')
run_comparison('Bru_Pot_Rainfed', model)

1272.2030826510295
12.722030826510295


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

36% RAW allowable depletion

In [33]:
# Loamy sand soil
loamy_sand = Soil(soil_type='LoamySand')

# crop
potato = Crop('PotatoLocalGDD', planting_date='04/25')

# create model with IrrMethod= Constant depth
model = AquaCropModel(sim_start_time=f'{1976}/04/25',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loamy_sand,
                      crop=potato,
                      irrigation_management=IrrigationManagement(irrigation_method=5,),
                      initial_water_content=fc,
                      )
                       

model._initialize()

while model._clock_struct.model_is_finished is False:    
    # get depth to apply, RAW = 36%, p_up2 = 0.6 so TAW = 21.6, inverse is 78.4
    depth=get_depth(model,78.4)
    
    model._param_struct.IrrMngt.depth=depth

    model.run_model(initialize_model=False)

# python_output(model, 'Bru_Pot_Irr36RAW')
run_comparison('Bru_Pot_Irr36RAW3', model)

1451.5172237974023
14.515172237974022


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

With irrigation type 1, testing

In [34]:
net_irr = IrrigationManagement(irrigation_method=1,SMT=[78.4]*4, MaxIrr=15)

model = AquaCropModel(sim_start_time=f'{1976}/04/25',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loamy_sand,
                      crop=potato,
                      irrigation_management=net_irr,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

run_comparison('Bru_Pot_Irr36RAW_noEToAdj', model)

1451.5172237974023
14.515172237974022


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

With irrigation type 4, testing

In [35]:
net_irr = IrrigationManagement(irrigation_method=4,NetIrrSMT=78.4, MaxIrr=15)

model = AquaCropModel(sim_start_time=f'{1976}/04/25',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loamy_sand,
                      crop=potato,
                      irrigation_management=net_irr,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

run_comparison('Bru_Pot_Irr36RAW3', model)

1451.379805796768
14.513798057967682


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

100% RAW

In [36]:
# create model with IrrMethod= Constant depth
model = AquaCropModel(sim_start_time=f'{1976}/04/25',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loamy_sand,
                      crop=potato,
                      irrigation_management=IrrigationManagement(irrigation_method=5,),
                      initial_water_content=fc,
                      )
                       

model._initialize()

while model._clock_struct.model_is_finished is False:    
    # get depth to apply, RAW = 100%, p_up2=0.6, TAW = 60%
    depth=get_depth(model,60)
    
    model._param_struct.IrrMngt.depth=depth

    model.run_model(initialize_model=False)

# python_output(model, 'Bru_Pot_Irr100RAW')
run_comparison('Bru_Pot_Irr100RAW3', model)

1451.5172237974023
14.515172237974022


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

With irrigation type 1, testing

In [37]:
net_irr = IrrigationManagement(irrigation_method=1,
                               SMT=[40,40,40,40], 
                               MaxIrr=15,
                            #    NetIrrSMT= 60,
                               )

model = AquaCropModel(sim_start_time=f'{1976}/04/25',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loamy_sand,
                      crop=potato,
                      irrigation_management=net_irr,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

run_comparison('Bru_Pot_Irr100RAW_noEToAdj', model)

1436.0493809723746
14.360493809723746


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

With irrigation type 4, testing

In [38]:
net_irr = IrrigationManagement(irrigation_method=4,
                               #SMT=[40,40,40,40], 
                               MaxIrr=15,
                               NetIrrSMT= 60,
                               )

model = AquaCropModel(sim_start_time=f'{1976}/04/25',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loamy_sand,
                      crop=potato,
                      irrigation_management=net_irr,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

run_comparison('Bru_Pot_Irr100RAW3', model)

1435.9937088655595
14.359937088655595


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

150% RAW

In [39]:
# create model with IrrMethod= Constant depth
model = AquaCropModel(sim_start_time=f'{1976}/03/01',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loamy_sand,
                      crop=potato,
                      irrigation_management=IrrigationManagement(irrigation_method=5,),
                      initial_water_content=fc,
                      )
                       

model._initialize()

while model._clock_struct.model_is_finished is False:    
    # get depth to apply, RAW = 150%, p_up2=0.6, TAW = 90%,0.9
    depth=get_depth(model,90)
    
    model._param_struct.IrrMngt.depth=depth

    model.run_model(initialize_model=False)

# python_output(model, 'Bru_Pot_Irr150RAW')
run_comparison('Bru_Pot_Irr150RAW3', model)

1451.5172237974023
14.515172237974022


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

With irrigation type 1, testing

In [40]:
net_irr = IrrigationManagement(irrigation_method=1,
                               SMT=[10]*4, 
                               MaxIrr=15,
                            #    NetIrrSMT= 60,
                               )

# Brussels climate
filepath=get_filepath('brussels_climate.txt')
brussels_weather = prepare_weather(filepath)

# Loamy sand soil
loamy_sand = Soil(soil_type='LoamySand')

# crops
potato = Crop('PotatoLocalGDD', planting_date='04/25')

# IWC
fc = InitialWaterContent(value=['FC'])


model = AquaCropModel(sim_start_time=f'{1976}/04/25',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loamy_sand,
                      crop=potato,
                      irrigation_management=net_irr,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

run_comparison('Bru_Pot_Irr150RAW_noEToAdj', model)

1397.853336725722
13.97853336725722


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

With irrigation type 4, testing

In [41]:
net_irr = IrrigationManagement(irrigation_method=4,
                            #    SMT=[10]*4, 
                               MaxIrr=15,
                               NetIrrSMT= 10,
                               )

model = AquaCropModel(sim_start_time=f'{1976}/04/25',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loamy_sand,
                      crop=potato,
                      irrigation_management=net_irr,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

run_comparison('Bru_Pot_Irr150RAW3', model)

1406.3612907895551
14.063612907895552


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

Get irrigation schedule from Windows

In [42]:
win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/Bru_Pot_Irr150RAW3WABAL.OUT', 
                            skiprows=4, skipfooter=27,  delim_whitespace=True,encoding="latin1")

     # WIN DAILY WABAL COLS = ['Day', 'Month', 'Year', 'DAP', 'Stage', 'WCTot', 'Rain', 'Irri', 'Surf',
    #    'Infilt', 'RO', 'Drain', 'CR', 'Zgwt', 'Ex', 'E', 'E/Ex', 'Trx', 'Tr',
    #    'Tr/Trx', 'ETx', 'ET', 'ET/ETx']
win_daily_water=win_daily_water.drop([0])
win_daily_water.rename(columns={
    'Year': 'year',
    'Month': 'month',
    'Day': 'day',
    },
    inplace=True, 
    errors='raise')
win_daily_water=remove_headers(win_daily_water)
win_daily_water['Irri']=win_daily_water['Irri'].astype(float)
win_irri = win_daily_water[win_daily_water['Irri']!=0.0].reset_index(drop=True)
win_irri['datetime']=pd.to_datetime(win_irri[['year', 'month', 'day']])

irri_dates = win_irri['datetime'].to_numpy()

n_years=len(irri_dates)
depths=[15]*n_years

schedule=pd.DataFrame([irri_dates,depths]).T # create pandas DataFrame
schedule.columns=['Date','Depth'] # name columns

C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\390516051.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/Bru_Pot_Irr150RAW3WABAL.OUT',


In [43]:
print(schedule)

          Date Depth
0   1976-06-23    15
1   1976-06-25    15
2   1976-06-27    15
3   1976-06-29    15
4   1976-07-01    15
..         ...   ...
144 2005-06-19    15
145 2005-06-20    15
146 2005-06-23    15
147 2005-06-25    15
148 2005-07-16    15

[149 rows x 2 columns]


In [44]:
irr_schedule = IrrigationManagement(irrigation_method=3, Schedule =schedule)

# Brussels climate
filepath=get_filepath('brussels_climate.txt')
brussels_weather = prepare_weather(filepath)

# Loamy sand soil
loamy_sand = Soil(soil_type='LoamySand')

# crops
potato = Crop('PotatoLocalGDD', planting_date='04/25')

# IWC
fc = InitialWaterContent(value=['FC'])


model = AquaCropModel(sim_start_time=f'{1976}/04/25',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loamy_sand,
                      crop=potato,
                      irrigation_management=irr_schedule,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Bru_Pot_Irr150RAW_noEToAdj')
run_comparison('Bru_Pot_Irr150RAW_noEToAdj', model)

1374.720832959984
13.74720832959984


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

## 9.6 Climate change impact

Historical:

In [45]:
# Hist climate
filepath=get_filepath('brussels_climate.txt')
brussels_weather = prepare_weather(filepath)

# Local Brussels soil
loam_soil = Soil(soil_type='Loam')

# crops
potato = Crop('PotatoLocalGDD', planting_date='04/25')

# IWC
fc = InitialWaterContent(value=['FC'])

# irr management
irr_mngt = IrrigationManagement(irrigation_method=0)

model = AquaCropModel(sim_start_time=f'{1976}/03/01',
                      sim_end_time=f'{2005}/12/31',
                      weather_df=brussels_weather,
                      soil=loam_soil,
                      crop=potato,
                      irrigation_management=irr_mngt,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Bru_Pot_Historical')
run_comparison('Bru_Pot_Historical3', model)

1390.9983956694628
13.909983956694628


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

Future:

In [46]:
# Future climate
filepath=get_filepath('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/aquacrop/aquacrop/data/brussels_future.txt')
brussels_future = prepare_weather(filepath)

# crops
potato = Crop('PotatoLocalGDD', planting_date='04/25')

model = AquaCropModel(sim_start_time=f'{2041}/04/25',
                      sim_end_time=f'{2070}/12/31',
                      weather_df=brussels_future,
                      soil=loam_soil,
                      crop=potato,
                      irrigation_management=irr_mngt,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Bru_Pot_Future')
run_comparison('Bru_Pot_Future3', model)

1597.8011883309516
15.978011883309517


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this

### Convert future climate data from AQ-Win formatting for use in AQ-Py

In [47]:
future_eto=pd.read_csv('../../AquaCrop-windows/AquaCropV7.1/DATA/BrusselsFuture2py.ETo')
future_temp=pd.read_table('../../AquaCrop-windows/AquaCropV7.1/DATA/BrusselsFuture2py.Tnx')
future_precip=pd.read_table('../../AquaCrop-windows/AquaCropV7.1/DATA/BrusselsFuture2py.PLU')

temp=future_eto.join(future_temp)
future_clim=temp.join(future_precip)
future_clim['datetime']=pd.date_range('2041/01/01','2070/12/31')
future_clim['Year']=pd.DatetimeIndex(future_clim['datetime']).year
future_clim['Month']=pd.DatetimeIndex(future_clim['datetime']).month
future_clim['Day']=pd.DatetimeIndex(future_clim['datetime']).day

future_clim.rename(columns={
    'Tmin (C)' : 'Tmin(C)',
    'TMax (C)' : 'Tmax(C)',
    'Average ETo (mm/day)' : 'Et0(mm)',
    'Total Rain (mm)' : 'Prcp(mm)'
},
inplace=True    
)

print(future_clim)

future_clim.drop('datetime',axis=1,inplace=True)
col_order=['Day', 'Month','Year','Tmin(C)','Tmax(C)','Prcp(mm)','Et0(mm)']
future_clim=future_clim[col_order]

future_clim.to_csv('brussels_future.txt', sep='\t')

print(future_clim)


       Et0(mm)  Tmin(C)  Tmax(C)  Prcp(mm)   datetime  Year  Month  Day
0          0.4      2.5      8.5       0.0 2041-01-01  2041      1    1
1          0.2      0.2      6.3       0.0 2041-01-02  2041      1    2
2          0.6     -1.4      7.9       0.0 2041-01-03  2041      1    3
3          0.5     -3.0      3.9       0.0 2041-01-04  2041      1    4
4          1.1    -10.6      6.1       0.0 2041-01-05  2041      1    5
...        ...      ...      ...       ...        ...   ...    ...  ...
10952      0.6      6.0     11.9       9.7 2070-12-27  2070     12   27
10953      0.4      0.4      8.3       0.0 2070-12-28  2070     12   28
10954      0.4      2.0      6.8       0.0 2070-12-29  2070     12   29
10955      0.3     -0.8      3.7       0.0 2070-12-30  2070     12   30
10956      0.3      3.3      6.6       0.0 2070-12-31  2070     12   31

[10957 rows x 8 columns]
       Day  Month  Year  Tmin(C)  Tmax(C)  Prcp(mm)  Et0(mm)
0        1      1  2041      2.5      8.5       0

# TEST: Setting up irrigation schedule to check it's working as expected

In [48]:
irri_dates = ['2000-04-26', '2000-04-27', '2000-04-28', '2000-04-30', '2000-05-10']
n_years=len(irri_dates)
depths=[21, 22, 23, 24, 25]

schedule=pd.DataFrame([irri_dates,depths]).T # create pandas DataFrame
schedule.columns=['Date','Depth'] # name columns

irr_schedule = IrrigationManagement(irrigation_method=3, Schedule =schedule)

# Brussels climate
filepath=get_filepath('brussels_climate.txt')
brussels_weather = prepare_weather(filepath)

# Loamy sand soil
loamy_sand = Soil(soil_type='LoamySand')

# crops
potato = Crop('PotatoLocalGDD', planting_date='04/25')

# IWC
fc = InitialWaterContent(value=['FC'])


model = AquaCropModel(sim_start_time=f'{2000}/04/25',
                      sim_end_time=f'{2000}/12/31',
                      weather_df=brussels_weather,
                      soil=loamy_sand,
                      crop=potato,
                      irrigation_management=irr_schedule,
                      initial_water_content=fc,
                      )
model.run_model(till_termination=True)

# python_output(model, 'Bru_Pot_Irr150RAW_noEToAdj')
run_comparison('Bru_Pot_Irr150RAW_noEToAdj', model)

1248.1736835111737
12.481736835111738


C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_outputs = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'Crop.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:74: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  win_daily_water = pd.read_table('C:/Users/s10034cb/Dropbox (The University of Manchester)/Manchester Postdoc/AquaCrop-windows/AquaCropV7.1/OUTP/'+win_filename+'WABAL.OUT',
C:\Users\s10034cb\AppData\Local\Temp\ipykernel_6868\341318044.py:166: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this